In [1]:
#Importing packages
import cobra
import cplex
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model
import pandas as pd

In [2]:
#Importing model iYali
yali = cobra.io.read_sbml_model('iYali-tidy v.5.xml')

In [3]:
threshold = 3;

In [4]:
yali.medium

{'y001654': 1000.0,
 'y001714': 1.0,
 'y001832': 1000.0,
 'y001861': 1000.0,
 'y001992': 1000.0,
 'y002005': 1000.0,
 'y002020': 1000.0,
 'y002049': 1000.0,
 'y002060': 1000.0,
 'y002100': 1000.0}

In [5]:
yali.reactions.y002060

Reaction identifier,y002060
Name,sulphate exchange
Memory address,0x01ea9ce48040
Stoichiometry,s_1468 <=> sulphate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [6]:
biomass = 'xBIOMASS'

In [9]:
selectedRxnList = ['y300083', 'y300084', 'y300085', 'y300086', 'y300087', 'y300088', 'y300089', 'y300090', 'y300091', 'y300092', 'y300093', 'y300094' 'y300095', 'y300096', 'y300097', 'y300098', 'y300100', 'y300101', 'y300102', 'y300103', 'y300104','y300105', 'y300106', 'y300107', 'y300108','y000028', 'y002229', 'y000252', 'y001674', 'y001191', 'y001673', 'y001882', 'y001120', 'y300094', 'y300095', 'y300096']

In [10]:
yali.reactions.y001714.lower_bound = -10.0
yali.reactions.y001714.lower_bound

-10.0

In [27]:
# *Aim:* *finding optKnock reactions sets of size 2 for increasing production 
# of succinate
# Set optKnock options
# The exchange of succinate will be the objective of the outer problem
options = struct('y300099', 'xBIOMASS', 'numDel', 2)
# We will impose that biomass be at least 50% of the biomass of wild-type
constrOpt = struct('rxnList', {{xBIOMASS}},'values', 0.5*fbaWT.f, 'sense', 'G')
# We will try to find 10 optKnock sets of a maximun length of 2
previousSolutions = cell(10, 1)
contPreviousSolutions = 1
nIter = 1
while nIter < threshold:
    fprintf('...Performing optKnock analysis...\n')
    if isempty(previousSolutions[1]):
        optKnockSol = OptKnock(yali, selectedRxnList, options, constrOpt)
    else:
        optKnockSol = OptKnock(yali, selectedRxnList, options, constrOpt, previousSolutions, 1)
    
    # determine succinate production and growth rate after optimization
    MargFluxM1 = optKnockSol.fluxes(strcmp(yali.rxns, 'y300099'))
    growthRateM1 = optKnockSol.fluxes(strcmp(yali.rxns, biomass))
    setM1 = optKnockSol.rxnList
    
    
    if ~isempty(setM1):
        previousSolutions[contPreviousSolutions] = setM1
        contPreviousSolutions = contPreviousSolutions + 1
        #printing results
        fprintf('optKnock found a optKnock set of large %d composed by ', length(setM1))
        a = length(setM1)
        for j in range (a):
            if j == 1:
                fprintf('%s', setM1[j])
            elif j == length(setM1):
                fprintf(' and %s', setM1[j])
            else:
                fprintf(', %s', setM1[j])

        fprintf('\n')
        fprintf('The production of succinate after optimization is %.2f \n', y300099)
        fprintf('The growth rate after optimization is %.2f \n', growthRateM1)
        fprintf('...Performing coupling analysis...\n')
        [type, maxGrowth, maxProd, minProd] = analyzeOptKnock(model, setM1, 'y300099')
        fprintf('The solution is of type: %s\n', type)
        fprintf('The maximun growth rate given the optKnock set is %.2f\n', maxGrowth)
        fprintf(['The maximun and minimun production of succinate given the optKnock set is %.2f and %.2f, respectively \n\n'], minProd, maxProd)
        if strcmp(type, 'growth coupled'):
            singleProductionEnvelope(model, setM1, 'EX_succ_e', biomass, 'savePlot', 1, 'showPlot', 1, 'fileName', ['y300099', num2str(nIter)], 'outputFolder', 'OptKnockResults')
    else:
        if nIter == 1:
            fprintf('optKnock was not able to found an optKnock set\n')
        else:
            fprintf('optKnock was not able to found additional optKnock sets\n')
        break
    nIter = nIter + 1

NameError: name 'struct' is not defined